## UNData API Exercise

In this exercise, you'll redo the data gathering phase of the UNData Exploration project by using APIs instead of downloading csv files.

You'll make use of the [World Bank Indicators API](https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation). Note that this API does not require an API key. Before attempting the exercise, it would be a good idea to skim through the Documentation page and to check out the [Basic Call Structure article](https://datahelpdesk.worldbank.org/knowledgebase/articles/898581).

In [36]:
import requests
import matplotlib.pyplot as plt
import json
import pandas as pd

### 1. Use the API to get all available data for the _GDP per capita, PPP (constant 2017 international $)_ indicator. Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [76]:
endpoint = 'https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD?' #add the json format 

params= {
    'Format': 'json',
    'Per_page': '20000', 
}

response = requests.get(endpoint, params=params)
print(response)
#assert str(print(response)) str.contains(200) #Create a assert statement that the reponse [200] and that it is valid
res = response.json()
len(res) #contains 2 lists

list_data= res[1]
len(list_data) #A list containing 50 dictionaries
gdp_ppp_data= pd.DataFrame(list_data)
gdp_ppp_data.head()

#clean up the dataframe 
clean_gdp_data= pd.json_normalize(list_data) #pull out the nested dictionary info 
clean_gdp_data= clean_gdp_data.explode('indicator.value')
clean_gdp_data['indicator.value'].unique()

clean_gdp_data.head()

<Response [200]>


,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2024,3968.963751,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
1,AFE,2023,3948.142721,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
2,AFE,2022,3974.244214,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
3,AFE,2021,3933.580905,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
4,AFE,2020,3861.068816,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern


### 2. Now, use the API to get all available data for _Life expectancy at birth, total (years)_. This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.

In [77]:
endpoint = 'https://api.worldbank.org/v2/country/all/indicator/SP.DYN.LE00.IN' #add the json format 

params= {
    'Format': 'json',
    'Per_page': '20000', 
}

response = requests.get(endpoint, params=params)
print(response)
#assert str(print(response)) str.contains(200) #Create a assert statement that the reponse [200] and that it is valid
res = response.json()
len(res) #contains 2 lists

list_data= res[1]
len(list_data) #A list containing 50 dictionaries
le_data= pd.DataFrame(list_data)

#clean up the dataframe 
le_data= pd.json_normalize(list_data) #pull out the nested dictionary info 
clean_le_data= clean_gdp_data.explode('indicator.value')
clean_le_data['indicator.value'].unique()

clean_le_data.head()

<Response [200]>


,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2024,3968.963751,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
1,AFE,2023,3948.142721,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
2,AFE,2022,3974.244214,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
3,AFE,2021,3933.580905,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
4,AFE,2020,3861.068816,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern


### 3. Merge the two results DataFrames together. You may want to rename or drop columns prior to merging.

In [67]:
le_formerge= clean_le_data[['date', 'value', 'country.value']]
le_formerge.columns= ['year', 'life expectancy', 'country']
le_formerge.head()

,year,life expectancy,country
0,2024,3968.963751,Africa Eastern and Southern
1,2023,3948.142721,Africa Eastern and Southern
2,2022,3974.244214,Africa Eastern and Southern
3,2021,3933.580905,Africa Eastern and Southern
4,2020,3861.068816,Africa Eastern and Southern


In [71]:
gdp_formerge= clean_gdp_data[['date', 'value', 'country.value']]
gdp_formerge.columns= ['year', 'GDP per capita', 'country']
gdp_formerge.head()

,year,GDP per capita,country
0,2024,3968.963751,Africa Eastern and Southern
1,2023,3948.142721,Africa Eastern and Southern
2,2022,3974.244214,Africa Eastern and Southern
3,2021,3933.580905,Africa Eastern and Southern
4,2020,3861.068816,Africa Eastern and Southern


In [73]:
gdp_le_data= pd.merge(le_formerge, gdp_formerge, on=['year', 'country'], how='inner') 
gdp_le_data.head()

,year,life expectancy,country,GDP per capita
0,2024,3968.963751,Africa Eastern and Southern,3968.963751
1,2023,3948.142721,Africa Eastern and Southern,3948.142721
2,2022,3974.244214,Africa Eastern and Southern,3974.244214
3,2021,3933.580905,Africa Eastern and Southern,3933.580905
4,2020,3861.068816,Africa Eastern and Southern,3861.068816


### 4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the [Country API](https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries). Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

In [95]:
endpoint=  'https://api.worldbank.org/v2/country'
params= {
    'Format': 'json',
    'Per_page': '20000' 
}

response = requests.get(endpoint, params=params)
print(response)
res = response.json()
list_data= res[1]
len(list_data)

list_data= res[1]
len(list_data) #A list containing 50 dictionaries
filter_data= pd.DataFrame(list_data)

# #clean up the dataframe 
# data_clean= pd.json_normalize(list_data) #pull out the nested dictionary info 
# data_cleana= data_clean.explode('indicator.value')
# data_clean['indicator.value'].unique()

# data_clean.head()
pd.json_normalize(filter_data)
filter_data

<Response [200]>


,id,iso2Code,name,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
0,ABW,AW,Aruba,"{'id': 'LCN', 'iso2code': 'ZJ', 'value': 'Lati...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'HIC', 'iso2code': 'XD', 'value': 'High...","{'id': 'LNX', 'iso2code': 'XX', 'value': 'Not ...",Oranjestad,-70.0167,12.5167
1,AFE,ZH,Africa Eastern and Southern,"{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': 'Aggregates'}",,,
2,AFG,AF,Afghanistan,"{'id': 'MEA', 'iso2code': 'ZQ', 'value': 'Midd...","{'id': 'MNA', 'iso2code': 'XQ', 'value': 'Midd...","{'id': 'LIC', 'iso2code': 'XM', 'value': 'Low ...","{'id': 'IDX', 'iso2code': 'XI', 'value': 'IDA'}",Kabul,69.1761,34.5228
3,AFR,A9,Africa,"{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': 'Aggregates'}",,,
4,AFW,ZI,Africa Western and Central,"{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': 'Aggregates'}",,,
...,...,...,...,...,...,...,...,...,...,...
291,XZN,A5,Sub-Saharan Africa excluding South Africa and ...,"{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': 'Aggregates'}",,,
292,YEM,YE,"Yemen, Rep.","{'id': 'MEA', 'iso2code': 'ZQ', 'value': 'Midd...","{'id': 'MNA', 'iso2code': 'XQ', 'value': 'Midd...","{'id': 'LIC', 'iso2code': 'XM', 'value': 'Low ...","{'id': 'IDX', 'iso2code': 'XI', 'value': 'IDA'}",Sana'a,44.2075,15.352
293,ZAF,ZA,South Africa,"{'id': 'SSF', 'iso2code': 'ZG', 'value': 'Sub-...","{'id': 'SSA', 'iso2code': 'ZF', 'value': 'Sub-...","{'id': 'UMC', 'iso2code': 'XT', 'value': 'Uppe...","{'id': 'IBD', 'iso2code': 'XF', 'value': 'IBRD'}",Pretoria,28.1871,-25.746
294,ZMB,ZM,Zambia,"{'id': 'SSF', 'iso2code': 'ZG', 'value': 'Sub-...","{'id': 'SSA', 'iso2code': 'ZF', 'value': 'Sub-...","{'id': 'LMC', 'iso2code': 'XN', 'value': 'Lowe...","{'id': 'IDX', 'iso2code': 'XI', 'value': 'IDA'}",Lusaka,28.2937,-15.3982


-------------

**Bonus Questions:** After doing a basic request to get all records, you can attempt these exercises to get additional practice using query parameters.

1. Adjust your request so that it returns data just for the United States.

2. Adjust your request so that it returns data just for the United States for the year 2021.

3. Adjust your request so that it returns data just for the United States for the years 2000 through 2021.

4. Adjust your request so that it returns data for the United States and Canada for the years 2000 through 2021.

5. If you haven't already done so and you would like to get some additional practice using loops, use the page parameter in order to pull all records. Do not change the value of the per_page parameter. You will likely need to utilize a loop of some kind in order to pull all records.

6. The endpoint "http://api.worldbank.org/v2/indicator" will return information about all available indicators. See [Indicator API Queries](https://datahelpdesk.worldbank.org/knowledgebase/articles/898599-indicator-api-queries). Use this endpoint to find the code for the "Public Expenditure on Education (% GDP)" indicator.